# Initial Python Setup

In [1]:
import pandas as pd
import numpy as np
import requests as req

from matplotlib import pyplot as plot

from zipfile import ZipFile
from io import BytesIO

# Setting Variables
Here we are just setting variables (such as the url, file name, etc) to tidy up the importing code

In [2]:
url = 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip'
file_name = 'survey_results_public.csv'

# Importing the Dataset

In [ ]:
response = req.get(url)
zip_file = ZipFile(BytesIO(response.content))
df = pd.read_csv(BytesIO(zip_file.read(file_name)))
display(df)